In [1]:
from etnn.tools.training_tools import ConfigStore
from etnn.routines.run_config import run_config

In [2]:
config = ConfigStore(
    in_dim=15,
    hidden_dim=128,
    out_dim=1,
    k=2,
    dataset=0,
    ds_size=10_000,
    num_gondolas=10,
    num_part_pg=5,
    loss_name='mse',
    optimizer_name='adam',
    num_max_epochs=100, # old: 20
    learning_rate=0.001,
    batch_size=1024,
    early_stop_tol=5,
    use_equal_batcher=False,
    seed=420,
    label_type="tree",  # tree
    final_label_factor=1/1000
)

In [3]:
label_list = ["tree", "default"]
k_list = [1, 2, 3, 4, 5]
counter = 0
total = len(label_list)

for label in label_list:
    config.label_type = label
    print(f"label: {label}")

    for k in k_list:
        config.k = k
        print(f"k:{k}")

        print(f"{counter}/{total}")
        counter += 1

        run_config(
            config=config,
            dataset_path="../datasets",
            results_folder="../results"
        )

label: tree
k:1
0/2


100%|██████████| 100/100 [05:42<00:00,  3.43s/it]


k:2
1/2
k:3
2/2


100%|██████████| 100/100 [05:44<00:00,  3.44s/it]


k:4
3/2


...

100%|██████████| 100/100 [05:51<00:00,  3.52s/it]


k:4
8/2


100%|██████████| 100/100 [05:45<00:00,  3.45s/it]


k:5
9/2


100%|██████████| 100/100 [07:36<00:00,  4.57s/it]
